In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from utils import save_config

pd.set_option('display.max_rows', 150)
ID = 1

In [2]:
rand_seed = 42

In [3]:
prj_folder = Path('.').resolve()
prj_folder

WindowsPath('C:/Users/pavel/OneDrive/Desktop/it_cup')

In [4]:
data = pd.read_csv(prj_folder/'2. Cup_IT_2022_Датасет_Data_Science.csv')

In [5]:
num_nans = data.isna().sum()
num_total = data.shape[0]
skip_rate = (num_nans / num_total * 100).sort_values(ascending=True)
keys = skip_rate[skip_rate < 50].keys()

In [6]:
data = data[keys]

In [7]:
X = data.iloc[:, 2:]
y = data.iloc[:, 1]

X = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=False, train_size=0.7, random_state=42)

In [8]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score
precision_score(y, np.zeros_like(y))

C:\Users\pavel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [55]:
rf_config = {
    'n_estimators': [200],
    #'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'max_depth' : [4,6],
    'criterion' :['entropy'],
    'class_weight': ['balanced']
}

gb_config = {
    'learning_rate': [0.1],
    'n_estimators': [200],
    'max_depth' : [3, 4, 5],
    'subsample': [1, 2]
}

scoring = ['accuracy', 'f1', 'roc_auc', 'precision', 'recall']

In [56]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#clf = RandomForestClassifier(random_state=rand_seed, n_jobs=8)
clf = GradientBoostingClassifier(random_state=rand_seed)

In [57]:
CUR_CONFIG = gb_config

save_config(ID, CUR_CONFIG, clf)
ID += 1

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=rand_seed)
validator = GridSearchCV(clf, param_grid=CUR_CONFIG, scoring=scoring, n_jobs=4, cv=cv, refit='f1', verbose=2)

In [58]:
validator.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


C:\Users\pavel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 18.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\pavel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pavel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Py

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=GradientBoostingClassifier(random_state=42), n_jobs=4,
             param_grid={'learning_rate': [0.1], 'max_depth': [3, 4, 5],
                         'n_estimators': [200], 'subsample': [1, 2]},
             refit='f1',
             scoring=['accuracy', 'f1', 'roc_auc', 'precision', 'recall'],
             verbose=2)

In [59]:
print(validator.best_estimator_)
print(validator.best_params_)
print(validator.best_score_)
print(validator.cv_results_)


GradientBoostingClassifier(max_depth=5, n_estimators=200, random_state=42,
                           subsample=1)
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1}
0.07611885235833338
{'mean_fit_time': array([8.33274900e+01, 2.62612502e-02, 1.11724215e+02, 2.85893281e-02,
       1.19413350e+02, 3.05844943e-02]), 'std_fit_time': array([4.56417328e-02, 4.97611088e-03, 9.61962142e-01, 1.69512369e-03,
       1.83467596e+01, 1.69560688e-03]), 'mean_score_time': array([0.12932086, 0.        , 0.14693952, 0.        , 0.12299863,
       0.        ]), 'std_score_time': array([0.00448657, 0.        , 0.00124273, 0.        , 0.01969008,
       0.        ]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 3, 4, 4, 5, 5],
             mask=[False, False, False, False, False, False],
       fill_va

In [60]:
from utils import fine_print
fine_print(validator.cv_results_)

m_acc:     [0.87390789        nan 0.8721309         nan 0.87126709        nan]
m_f1:      [0.04234776        nan 0.05610337        nan 0.07611885        nan]
m_rauc:    [0.70392533        nan 0.69856649        nan 0.69568897        nan]
m_prec:    [0.47226879        nan 0.39291448        nan 0.39003742        nan]
m_recall:  [0.02217395        nan 0.03021951        nan 0.04219011        nan]


In [61]:
from sklearn.metrics import confusion_matrix
preds = validator.predict(X_val)
confusion_matrix(y_val, preds)

array([[86675,   546],
       [ 7137,   185]], dtype=int64)

In [62]:
print(accuracy_score(y_val, preds))
print(f1_score(y_val, preds))
print(roc_auc_score(y_val, preds))
print(precision_score(y_val, preds))
print(recall_score(y_val, preds))

0.9187353902457083
0.04594561033155346
0.5095031801733781
0.253077975376197
0.025266320677410543


In [63]:
validator.best_estimator_.feature_importances_

array([0.01901826, 0.00848535, 0.02401716, 0.00706825, 0.01435461,
       0.02734047, 0.01321548, 0.01740449, 0.00858931, 0.0181361 ,
       0.00682897, 0.0158498 , 0.01077052, 0.0198339 , 0.00687223,
       0.02302185, 0.0134584 , 0.02150689, 0.02152631, 0.01860141,
       0.01986851, 0.02318572, 0.14186042, 0.0096727 , 0.02235251,
       0.02032245, 0.02251129, 0.00742925, 0.02660816, 0.02328255,
       0.00585369, 0.00752194, 0.0179913 , 0.00338755, 0.01363195,
       0.0173404 , 0.01963741, 0.01915202, 0.01827338, 0.02258011,
       0.01603749, 0.01502496, 0.01641252, 0.03154652, 0.0056824 ,
       0.02296905, 0.00954763, 0.02385728, 0.03611472, 0.01345115,
       0.0051739 , 0.02581928])